## Rentabilidades de los Fondos de Inversión Colectiva (FIC)

La rentabilidad de los Fondos de Inversión Colectiva (en adelante FIC) y Fondos de Capital Privado (en adelante FCP) está determinada por la variación del valor de unidad diaria de las unidades de inversión llamadas "Participaciones".

**URL Datos Abiertos:** https://www.datos.gov.co/Hacienda-y-Cr-dito-P-blico/Rentabilidades-de-los-Fondos-de-Inversi-n-Colectiv/qhpu-8ixx


**url_base:** https://www.datos.gov.co/resource/qhpu-8ixx.json

Notas:

https://github.com/trallard
https://docs.microsoft.com/en-us/windows/python/web-frameworks
https://medium.com/@tonybolanyo/variables-de-entorno-con-dotenv-en-python-4697540bf7e4

https://dev.socrata.com/foundry/www.datos.gov.co/qhpu-8ixx
https://dev.socrata.com/docs/queries/
https://dev.socrata.com/consumers/getting-started.html

https://dev.socrata.com/blog/2014/1/04/data-visualization-with-python.html


In [ ]:
# - Instalar paquetes en jupyter mac
# - !/usr/local/Cellar/jupyterlab/3.0.12/libexec/bin/python3.9 -m pip install pandas
# - !/usr/local/Cellar/jupyterlab/3.0.12/libexec/bin/python3.9 -m pip install python-dotenv

# - !/usr/local/Cellar/jupyterlab/3.0.12/libexec/bin/python3.9 -m pip show

In [2]:
import requests
import pandas as pd
from dotenv import find_dotenv, load_dotenv
import logging
import os

# --------------------------
# Métodos auxiliares - Helper methods - 
# --------------------------

logging.getLogger().setLevel(logging.INFO)
try:
    dotenv_path = find_dotenv('storage/keys/.env')
    logging.info('Dotenv localizado!!!')
    load_dotenv(dotenv_path)

except:
    logging.info("No hay variables locales")
        


INFO:root:Dotenv localizado!!!


In [3]:
# - Conexión a datos abiertos
url = 'https://www.datos.gov.co/resource/qhpu-8ixx.json'
headers = {
  'X-App-Token':os.environ.get('X_APP_TOKEN')
  }

response = requests.get(url,headers=headers)
response.status_code

200

In [4]:
os.environ

environ{'TMPDIR': '/var/folders/w2/767gjq293f12kc22wn6ccqrm0000gn/T/',
        '__CFBundleIdentifier': 'com.apple.Terminal',
        'XPC_FLAGS': '0x0',
        'TERM': 'xterm-color',
        'SSH_AUTH_SOCK': '/private/tmp/com.apple.launchd.IdjXFJ6ELT/Listeners',
        'XPC_SERVICE_NAME': '0',
        'TERM_PROGRAM': 'Apple_Terminal',
        'TERM_PROGRAM_VERSION': '440',
        'TERM_SESSION_ID': '204A4414-AC0E-4456-B3FB-884F43BE0018',
        'SHELL': '/bin/zsh',
        'HOME': '/Users/aheanor',
        'LOGNAME': 'aheanor',
        'USER': 'aheanor',
        'PATH': '/Library/Frameworks/Python.framework/Versions/3.9/bin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin:/Library/Apple/usr/bin',
        'SHLVL': '1',
        'PWD': '/Users/aheanor',
        'OLDPWD': '/Users/aheanor',
        'ZSH': '/Users/aheanor/.oh-my-zsh',
        'PAGER': 'cat',
        'LESS': '-R',
        'LSCOLORS': 'Gxfxcxdxbxegedabagacad',
        'LC_CTYPE': 'UTF-8',
        '_': '/usr/local/bin/jupyter',

In [28]:
import requests
import pandas as pd

host_FIC = 'https://www.datos.gov.co/resource/qhpu-8ixx.json?fecha_corte=2016-01-01T00:00:00.000'

response = requests.get(host_FIC)
dataList = response.json()

df = pd.DataFrame(dataList)
df.shape

(471, 26)

In [9]:
df.head()

,fecha_corte,tipo_entidad,nombre_tipo_entidad,codigo_entidad,nombre_entidad,tipo_negocio,nombre_tipo_patrimonio,subtipo_negocio,nombre_subtipo_patrimonio,codigo_negocio,...,valor_unidad_operaciones,aportes_recibidos,retiros_redenciones,anulaciones,valor_fondo_cierre_dia_t,numero_inversionistas,rentabilidad_diaria,rentabilidad_mensual,rentabilidad_semestral,rentabilidad_anual
0,2021-03-11T00:00:00.000,5,SF-SOCIEDAD FIDUCIARIA,62,Fiduciaria Coomeva S.A.,9,FONDOS DE INVERSIÓN COLECTIVA,1,FIC DE TIPO GENERAL,82379,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2021-03-11T00:00:00.000,5,SF-SOCIEDAD FIDUCIARIA,3,Bbva Asset Management S.A. Sociedad Fiduciaria,9,FONDOS DE INVERSIÓN COLECTIVA,1,FIC DE TIPO GENERAL,8686,...,24302.192417,0.590000,0.000000,0.000000,30144702392.100000,22.000000,28.784935,-5.571943,2.069361,6.126540
2,2021-03-11T00:00:00.000,5,SF-SOCIEDAD FIDUCIARIA,3,Bbva Asset Management S.A. Sociedad Fiduciaria,9,FONDOS DE INVERSIÓN COLECTIVA,1,FIC DE TIPO GENERAL,8686,...,24132.311896,33594003.230000,572823767.900000,0.000000,206727150316.110000,2326.000000,27.897297,-6.223332,1.365411,5.394686
3,2021-03-11T00:00:00.000,5,SF-SOCIEDAD FIDUCIARIA,3,Bbva Asset Management S.A. Sociedad Fiduciaria,9,FONDOS DE INVERSIÓN COLECTIVA,1,FIC DE TIPO GENERAL,8734,...,2636.563207,1867437001.740000,5085042766.000000,0.000000,47833818074.770000,19.000000,3.925802,0.354870,1.640968,3.504053
4,2021-03-11T00:00:00.000,5,SF-SOCIEDAD FIDUCIARIA,3,Bbva Asset Management S.A. Sociedad Fiduciaria,9,FONDOS DE INVERSIÓN COLECTIVA,1,FIC DE TIPO GENERAL,8734,...,2566.733371,1541482230.790000,1554231194.950000,0.000000,331460068654.590000,8861.000000,3.515523,-0.041349,1.239687,3.095437
